In [ ]:
!pip3 install torch
!pip3 install torchvision
!pip3 install torchaudio
!pip install comet_ml


     |████████████████████████████████| 3.2MB 4.6MB/s 
     |████████████████████████████████| 215kB 4.6MB/s 
     |████████████████████████████████| 204kB 13.4MB/s 
     |████████████████████████████████| 419kB 14.6MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=502b18e21a1f14961c373d19629bfcd029ecbea6b906cdd545f64285d279e745
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [ ]:
import os
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
import pandas as pd
import librosa

## ETL process

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path_audio= 'drive/My Drive/Speech Recognition project/recordings/'
path_text = 'drive/My Drive/Speech Recognition project/wolof_text/'
wav_text = 'drive/My Drive/Speech Recognition project/Wavtext_dataset2.csv'

## Data preparation for created the char.txt file from my dataset.

In [ ]:
wavtext = pd.read_csv(wav_text)
wavtext.head()

,0,text_path,wav_path,bloc,line,text
0,/storage/emulated/0/ligaikuma/examples/bloc10....,bloc10.txt (line 1),200702-231930_wol_8ec_elicit_0.wav,bloc10.txt,1,Nu ngi indee woon jiwu ji sapoN
1,/storage/emulated/0/ligaikuma/examples/bloc10....,bloc10.txt (line 2),200702-231930_wol_8ec_elicit_1.wav,bloc10.txt,2,Nu ngi tollu dagana
2,/storage/emulated/0/ligaikuma/examples/bloc10....,bloc10.txt (line 3),200702-231930_wol_8ec_elicit_2.wav,bloc10.txt,3,Nu ngi wàkkirlu ci yàlla
3,/storage/emulated/0/ligaikuma/examples/bloc10....,bloc10.txt (line 4),200702-231930_wol_8ec_elicit_3.wav,bloc10.txt,4,Nu ñow nag def ko ci kër gi
4,/storage/emulated/0/ligaikuma/examples/bloc10....,bloc10.txt (line 5),200702-231930_wol_8ec_elicit_4.wav,bloc10.txt,5,Nuggaayu néeg bee ko fi téye


In [ ]:
df = wavtext[['wav_path','text']]
df['wav_path'] = df['wav_path'].apply(lambda x: x.replace(' ',''))
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,wav_path,text
0,200702-231930_wol_8ec_elicit_0.wav,Nu ngi indee woon jiwu ji sapoN
1,200702-231930_wol_8ec_elicit_1.wav,Nu ngi tollu dagana
2,200702-231930_wol_8ec_elicit_2.wav,Nu ngi wàkkirlu ci yàlla
3,200702-231930_wol_8ec_elicit_3.wav,Nu ñow nag def ko ci kër gi
4,200702-231930_wol_8ec_elicit_4.wav,Nuggaayu néeg bee ko fi téye


In [ ]:
from glob import glob
lastpath = 'drive/My Drive/Speech Recognition project/data/last_records/'
audioPath = glob(lastpath+"*/*.wav*", recursive=True)
audioPath[0:5]

['drive/My Drive/Speech Recognition project/data/last_records/200705-180253_wof_758_elicit/200705-180253_wof_758_elicit_0.wav',
 'drive/My Drive/Speech Recognition project/data/last_records/200705-180253_wof_758_elicit/200705-180253_wof_758_elicit_1.wav',
 'drive/My Drive/Speech Recognition project/data/last_records/200705-180253_wof_758_elicit/200705-180253_wof_758_elicit_2.wav',
 'drive/My Drive/Speech Recognition project/data/last_records/200705-180253_wof_758_elicit/200705-180253_wof_758_elicit_3.wav',
 'drive/My Drive/Speech Recognition project/data/last_records/200705-180253_wof_758_elicit/200705-180253_wof_758_elicit_5.wav']

In [ ]:
import shutil 
from shutil import  copy

In [ ]:
for wav in audioPath:
  copy(wav,'drive/My Drive/Speech Recognition project/all_audios/')


In [ ]:
wavpath = '../drive/My Drive/Speech Recognition project/all_audios/'

In [ ]:
np.random.seed(0)
indices = np.random.permutation(len(df['wav_path']))

test_idx = indices[:len(indices)//2]
validation_portion = int(len(indices)//2 * 0.2)
train_idx = indices[len(indices)//2:-validation_portion]
valid_idx = indices[-validation_portion:]

In [ ]:
datapath = 'drive/My Drive/Speech Recognition project/data/records'

251

In [ ]:
trainpath = '../drive/My Drive/Speech Recognition project/data/records/train/'
valpath =  '../drive/My Drive/Speech Recognition project/data/records/val/'
testpath =  '../drive/My Drive/Speech Recognition project/data/records/test/'


In [ ]:
train_set_files = [df['wav_path'].iloc[i] for i in train_idx]
valid_set_files = [df['wav_path'].iloc[i] for i in valid_idx]
test_set_files = [df['wav_path'].iloc[i] for i in test_idx]

In [ ]:
for wav in train_set_files:
  fil = wavpath+wav
  if os.path.exists(fil):
    copy(wavpath+wav,trainpath)

for wav in valid_set_files:
  fil = wavpath+wav
  if os.path.exists(fil):
    copy(wavpath+wav,valpath)

for wav in test_set_files:
  fil = wavpath+wav
  if os.path.exists(fil):
    copy(wavpath+wav,testpath)

In [ ]:
import re
char_idx = 2
chars = {" ":1}
df['text']=df['text'].apply(lambda x: re.sub("[-!;»,_...«.\"\"\(\)~#:?…“”–]","",x))
for line in df['text']:
  #line = re.sub("[-!;»,_...«.\"\"\(\)~#:?…“”–]","",line)
  char_set = set(line)
  for c in char_set:
      if c not in chars:
          chars[c]=char_idx
          char_idx+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
indices_text = []
for line,wav in zip(df['text'],df['wav_path']):
    line = list(line)
    indicies = []
    wav = wav.split('.')[0]
    for c in line:
        indicies.append(str(chars[c]))
    indices_text.append(wav+" "+" ".join(indicies))

In [ ]:
indicies_text = "\n".join(indices_text)
with open("drive/My Drive/Speech Recognition project/data/chars2.txt","w") as f:
    f.write(indicies_text)